# Examen

In [71]:
#Numpy y pandas
import numpy as np
import pandas as pd

#Conjunto de datos
from sklearn.datasets import load_wine

#Validacion cruzada
from sklearn.model_selection import StratifiedKFold

#Normalizacion de datos
from sklearn.preprocessing import MinMaxScaler

#Algoritmo K-Medias
from sklearn.cluster import KMeans

#Distancia euclidea
from sklearn.metrics.pairwise import euclidean_distances

#Moda
from scipy.stats import mode

#MLP
from sklearn.neural_network import MLPClassifier

#Metrics - Tasa de acierto
from sklearn.metrics import accuracy_score

### Cargamos los datos

In [72]:
datos = load_wine()
datos

#Creamos un DataFrame para visualizar mejor la informacion
df = pd.DataFrame(data=datos.data, columns=datos.feature_names)

#Añadimos la clase al final
df.insert(loc=len(df.columns), column='Class', value=datos.target)
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,Class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


### Escalado de los datos
Mediante el escalado lo que hacemos es llevar todos los atributos al rango [0,1], para darles a todos el mismo peso

In [73]:
X = MinMaxScaler().fit_transform(datos.data)
y = datos.target

### RBF

Las capas de funciones de base radial estas compuestas por 3 capas: una capa de entrada, una única capa oculta y una capa de salida. El primer parametro que nos aparece es el numero de neuronas. La capa oculta se entrena mediante el algoritmo de las K-Medias, el cual requiere el numero de cluster. Por ello, consideramos el numero de cluster igual al numero de neuronas

In [74]:
#Validacion Cruzada (k = 10)
K = 10
kfold = StratifiedKFold(n_splits=K)

#Valores min y max del cluster
           
min_clusters = 1
max_clusters = 10

#DataFrame en el que se almacena la informacion
tabla = pd.DataFrame(columns=['n_clusters', 'tasa_acierto'])

#Tabla Completa que almacenara todas las tasas de error
cabecera = ['n_clusters 1', 'n_clusters 2','n_clusters 3', 'n_clusters 4','n_clusters 5', 'n_clusters 6','n_clusters 7', 'n_clusters 8','n_clusters 9', 'n_clusters 10']
tablaCompleta = pd.DataFrame(columns=cabecera)


for c in range(min_clusters, max_clusters+1):
    
    #Agrupamiento
    kmedias = KMeans(n_clusters=c, random_state=0)
    tasa = 0.0
    
    for train_index, test_index in kfold.split(X, y):
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        #Entrenamiento
        kmedias.fit(x_train, y_train)
        
        #Metodo de la moda
        moda = np.zeros(c, dtype=int)
        
        for i in range(c):
            moda[i] = mode(y_train[np.argwhere(kmedias.labels_ == i)])[0].astype(int)
            
        #Prediccion
        y_predict = moda[kmedias.predict(x_test)]
        
        tasa += accuracy_score(y_test, y_predict)
        
        #Escribimos las tasas de error
        tablaCompleta.loc[c,:] = [accuracy_score(y_test, y_predict)]
    
    tasa /= 10
    tabla.loc[c] = [c, tasa]

#Mostramos los resultados
tabla

,n_clusters,tasa_acierto
1,1.0,0.399346
2,2.0,0.600654
3,3.0,0.938562
4,4.0,0.966667
5,5.0,0.949346
6,6.0,0.938235
7,7.0,0.937908
8,8.0,0.955229
9,9.0,0.944118
10,10.0,0.943791


### Cluster optimo y maxima tasa de acierto

In [75]:
cluster_optimo = tabla.tasa_acierto.argmax()
print("El cluster optimo es = ", cluster_optimo+min_clusters, " con tasa de acierto = ", tabla.tasa_acierto.iloc[cluster_optimo])

El cluster optimo es =  4  con tasa de acierto =  0.9666666666666668


### RBF + MLP

Una vez hallado el cluster optimo, entrenamos la RBF con el algoritmo K-Media. Ahora, la capa de salida la entrenaremos mediate MLP.

In [76]:
tablaFinal = pd.DataFrame(columns=['K', 'tasa_acierto'])
index = 0

for train_index, test_index in kfold.split(X, y):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    #Agrupamiento
    kmedias = KMeans(n_clusters=cluster_optimo+min_clusters, random_state=0)
    #Entrenamienta
    kmedias.fit(x_train, y_train)
    
    #Creacion de centros
    centros = kmedias.cluster_centers_
    
    #Matriz de distancias - Sigma
    distancias = euclidean_distances(centros)
    
    sigma = np.sum(distancias, axis=0) / (centros.shape[0]-1)
    
    #Salida capa oculta - Datos entrenamiento
    x_mlp_train = np.zeros((x_train.shape[0], centros.shape[0]), dtype=float)
    for i in range(x_train.shape[0]):
        for j in range(centros.shape[0]):
            x_mlp_train[i][j] = np.exp(-np.sum((x_train[i] - centros[j])**2) / (2.0*(sigma[j]**2)))
    
    #Salida capa oculta - Datos test
    x_mlp_test = np.zeros((x_test.shape[0], centros.shape[0]), dtype=float)
    for i in range(x_test.shape[0]):
        for j in range(centros.shape[0]):
            x_mlp_test[i][j] = np.exp(-np.sum((x_test[i] - centros[j])**2) / (2.0*(sigma[j]**2)))
            
            
    #MLP
    #Creacion del modelo
    clf = MLPClassifier(hidden_layer_sizes=(10,), max_iter=5000)
    #Entrenamiento
    clf.fit(x_mlp_train, y_train)
    #Prediccion
    y_predic = clf.predict(x_mlp_test)
    
    #Tasa de acierto
    tablaFinal.loc[index] = [index, accuracy_score(y_test, y_predic)]
    index += 1
    
    
#Tasas de acierto con MLP
tablaFinal

,K,tasa_acierto
0,0.0,0.944444
1,1.0,1.000000
2,2.0,0.944444
3,3.0,0.944444
4,4.0,0.944444
5,5.0,1.000000
6,6.0,1.000000
7,7.0,0.944444
8,8.0,1.000000
9,9.0,1.000000
